# Data 전처리

In [2]:
log_file = './log_openai_compat_a.x_10user.out'

In [9]:
import pandas as pd
log_df = pd.read_csv(log_file, delimiter='\|\|', names=['epoch_time', 'user_Id', 'message1', 'message2'])
sorted_df = log_df.sort_values(by=['user_Id', 'epoch_time'])
sorted_df

C:\Users\mjher\AppData\Local\Temp\ipykernel_21492\1374128219.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  log_df = pd.read_csv(log_file, delimiter='\|\|', names=['epoch_time', 'user_Id', 'message1', 'message2'])


,epoch_time,user_Id,message1,message2
44562,1718082988351,01s9mfd5hpn1fxvf,Start to inference,Prompt : What company makes the objectively b...
44599,1718082988472,01s9mfd5hpn1fxvf,Token received :,None
44602,1718082988501,01s9mfd5hpn1fxvf,Token received : That,None
44612,1718082988532,01s9mfd5hpn1fxvf,Token received : 's,None
44622,1718082988565,01s9mfd5hpn1fxvf,Token received : what,None
...,...,...,...,...
24335,"참 재미있고, 유익한 이야기 입니다.",None,None,None
24328,행복한 삶을 살게 됩니다.,None,None,None
127312,👤 Eyewitness News Staff,None,None,None
127313,"📅 Published on January 2, 2019 🕒 4 min read",None,None,None
